In [1]:
import dg
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt
def imshow(data):
    img = data / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(npimg.reshape([28,28]), cmap = 'gray')
    plt.show()
    
def label2onehot(labels):
    b_size = len(labels)
    ret = np.zeros([b_size, 10, 1])
    for i, label in enumerate(labels):
        ret[i][label] = 1
    return ret

In [2]:
batch_size = 15
num_labels = 10

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])


In [4]:
trainset = torchvision.datasets.MNIST(root = './data', train = True,
                                     download = True, transform=transform)

In [5]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [6]:
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                    download=True, transform=transform)

In [7]:
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [10]:
w1_stdev = 0.1
w2_stdev = 0.01

x = dg.identity(np.random.randn(batch_size, 784, 1), name = 'x')
w_1 = dg.identity(np.random.randn(500, 784) * w1_stdev, name = 'w_1')
b_1 = dg.identity(np.random.randn(500, 1) * 1, name = 'b_1')
w_2 = dg.identity(np.random.randn(10, 500) * w2_stdev, name = 'w_2')
b_2 = dg.identity(np.random.randn(10, 1) * 1, name = 'b_2')
y_label = dg.identity(np.random.randn(batch_size, 10, 1), name = 'ylabel')

y_1 = dg.mat_add(dg.mat_mul(w_1, x, 'mat_mul_1'), b_1, name = 'y_1')
y_1 = dg.sigmoid(y_1, name = 'sigmoid')
y_2 = dg.mat_add(dg.mat_mul(w_2, y_1, 'mat_mul_2'), b_2, name = 'y_2')

loss = dg.softmax_cross(y_2, y_label, name = 'softmax_loss')


In [13]:
lr = 0.001
sgd_optim = dg.optim.SGD(loss, [w_1, w_2, b_1, b_2], lr)
epoch = 5
for ep in range(epoch):
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs = inputs.reshape([batch_size, 784, 1]).numpy()
        labels = label2onehot(labels)
        sgd_optim.step({x:inputs, y_label:labels})
        if i % 100 == 0:
            print('Loss after training {} times: {}'.format(i+1, loss.data()))

Loss after training 1 times: [12.20760686]
Loss after training 101 times: [2.38701455]
Loss after training 201 times: [2.35296852]
Loss after training 301 times: [2.29710349]
Loss after training 401 times: [2.4435143]
Loss after training 501 times: [2.82936918]
Loss after training 601 times: [2.5464516]
Loss after training 701 times: [2.21154457]
Loss after training 801 times: [2.56770033]
Loss after training 901 times: [2.27376907]
Loss after training 1001 times: [2.57240702]
Loss after training 1101 times: [3.127169]
Loss after training 1201 times: [2.38224515]
Loss after training 1301 times: [2.46754913]
Loss after training 1401 times: [2.5978583]
Loss after training 1501 times: [2.49556048]
Loss after training 1601 times: [2.500571]
Loss after training 1701 times: [2.61402396]
Loss after training 1801 times: [2.55650662]
Loss after training 1901 times: [2.51251163]
Loss after training 2001 times: [2.53256177]
Loss after training 2101 times: [2.34243008]
Loss after training 2201 tim

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/liyiqing/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/liyiqing/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/liyiqing/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/liyiqing/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/liyiqing/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/liyiqing/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/liyiqing/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_by